This notebook is the result of issue [#43](https://github.com/alliander-opensource/AIFES/issues/43)

# Sum of forecasts or forecast the sum?
Decisions are seldom based on forecasts for single sensors. Therefore combining either sensor data or the forecasts of these timeseries is nescesarry to obtain forecasts for the quantitiy of interest. Various strategies can be employed to achieve this with each of them having advantages and disadvantages. 

Here, we compare these strategies to forecast the total load on a substation: 
1) Forecast the total load directly.
2) Forecast and combine:
   1) Total load large customers
   2) Residual load substation
3) Forecast and combine
   1) Individual load large customers
   2) Residual load substation

We are going to run this comparison for Westwoud, since there the customer population is most diverse. To generate forecasts, we use the openSTEF backtest pipeline. Because of the stochastic nature of the backtest we repeat it 10 times to aquire some statistics. We compare the resulting forecasts in terms of MAE, rMAE and rMAE for the lowest 5% of the values.

# Imports and data preparation

## Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from pathlib import Path
from datetime import datetime
import os

from openstef.pipeline.train_create_forecast_backtest import train_model_and_forecast_back_test
from openstef.metrics.figure import plot_feature_importance
from openstef.data_classes.model_specifications import ModelSpecificationDataClass
from openstef.data_classes.prediction_job import PredictionJobDataClass

# Set working dir to location of this file
os.chdir('.')

# Set plotly as the default pandas plotting backend
pd.options.plotting.backend = 'plotly'

In [ ]:
import plotly.io as pio

# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "plotly_mimetype+notebook"

## EMS measurements
Load, pre-process, and visualize 

In [ ]:
# Load inputs
filename = Path("../.data/Westwoud-50_10kV.csv")

measurements = pd.read_csv(filename, delimiter=";", decimal=",")
measurements["Datetime"] = pd.to_datetime(measurements.iloc[:,0] + " " + measurements.iloc[:,1])
measurements = measurements.set_index('Datetime').tz_localize('CET', ambiguous='NaT', nonexistent='NaT').tz_convert("UTC")

# Only keep relevant columns
measurements = measurements.iloc[:,2:-1]

# Sum the load
measurements['Total'] = measurements.sum(axis=1)

# By default, only a backtest is made for the total
target_column = 'Total'

# Drop all rows with a NaT index.
measurements = measurements[measurements.index.notna()]

# remove dupicates
measurements = measurements[~measurements.index.duplicated()]

# Resample per 15 minutes
measurements = measurements.resample('15T').mean()

# Polarity of measurements is inverted
measurements *= -1

measurements.iloc[1000:2000,:].plot()

### Check the validity of the measurements

In [ ]:
# Validate that there are no duplicates left
assert not(measurements.index.duplicated().any()), "Duplicate indices have been found in the measurements dataframe."

## Measurements large customers (C-ARM data)
Load, pre-process, and visualize  

In [ ]:
large_clients = pd.read_csv("../.data/westwoud_clients.csv", index_col=0)

carm_measurements = pd.read_csv("../.data/wew_customer_carm_measurements.csv", 
                                delimiter=",", decimal=".", index_col=0, parse_dates=True)
carm_measurements /= 1E6 # rescale from -W to to MW

## Let's inspect the measurements of large customers and determine their contribution to the total load

In [ ]:
import numpy as np
carm_measurements.aggregate([max, np.median, min]).T.plot(kind='box')

In [ ]:
carm_measurements['Total'] = carm_measurements.sum(axis=1)
carm_measurements.iloc[20000:21000].plot()

In [ ]:
total_customer_load = carm_measurements['Total']
residual_load = pd.DataFrame(dict(load=measurements[target_column] - total_customer_load))
# for sanity, plot the total, customer and residual load
loads = measurements[[target_column]].merge(pd.DataFrame(residual_load), left_index=True, right_index=True, how='outer').merge(pd.DataFrame(total_customer_load), left_index=True, right_index=True, how='outer')
loads.columns=['Realized','Residual','Customer_Total']

loads.iloc[1000:2000,:].plot()

## Change of plans
We find large differences between the total load, and the Customer_total. Let's change our original plan and asses:
Investigate difference in accuracy:
- Forecast `customer_total` directly
- Forecast each customer seperately

In [ ]:
# Find largest customers:
largest_customers = carm_measurements.abs().max(axis=0).sort_values(ascending=False)[:6]
carm_measurements.loc[:,largest_customers.index].iloc[21000:22000].plot()
# Looks like 1 windpark, 2 solar parks, and 2 large consumers

The Figure shows the timeseries of the Total customer load, and the 5 largest customers. These include a windpark, two solar parks and two large consumers. The windpark and one of the solar parks are an order of magnitude larger than the next largest customers

## Predictors
Load, pre-process, and visualize 

In [ ]:
# Load predictors
predictors = pd.read_csv('../.data/weather_apx_sji_sja_Middenmeer.csv', index_col=0, parse_dates=True)
predictors.head()

In [ ]:
# Check the validity of the predictors data
assert not(predictors.duplicated().any()), "Duplicate values have been found in the predictors dataframe."
assert not(predictors.index.duplicated().any()), "Duplicate indices have been found in the predictors dataframe."

# Backtests

#### Configure training, prediction, and backtest specifications

In [ ]:
# Define properties of training / prediction. We call this a 'prediction_job'.
pj=PredictionJobDataClass(
    id=1, # Does not matter in a backtest context
    name='TestPrediction', # Does not matter in a backtest context
    model='xgb',
    quantiles=[0.5], # We will only consider the P50 forecast
    horizon_minutes=24*60,
    resolution_minutes=15,
    forecast_type="demand", # Note, this should become optional
    lat = 1, # should become optional
    lon = 1, # should become optional
)

# The modelspecs do not do much if only an "id" is specified.
modelspecs = ModelSpecificationDataClass(id=pj['id'])

# Define backtest specs.
backtest_specs = dict(n_folds=3, 
                      # The training horizon also decides for which forecast horizon, backtest forecasts are made.
                      training_horizons=[24.0]) # We will only consider a single training horizon

## Forecast Total Customers Directly

In [ ]:
# OpenSTEF always expects a column called "load". This is the column it will predict.
load = pd.DataFrame(dict(load=carm_measurements['Total']))
input_data = load.merge(predictors, left_index=True, right_index=True, how='inner')

assert not(input_data.index.duplicated().any()), "There are duplicate indices in the input data."

#### Perform and save the results of the backtest `n_iterations` times

In [ ]:
from utils.persisting_artifacts import write_artifacts

In [ ]:
n_iterations = 1 #set to a small number while developing, set to 10 for the Champion Data

In [ ]:
for i in range(n_iterations):
    # Perform the backtest
    forecast, models, train_data, validation_data, test_data = train_model_and_forecast_back_test(
        pj,
        modelspecs = modelspecs,
        input_data = input_data,
        **backtest_specs,
    )
    
    # If n_folds > 1, models is a list of models. In that case, only use the first model.
    if backtest_specs['n_folds'] > 1:
        model=models[0]
    else:
        model=models

    run_name = f"{datetime.utcnow():%Y%m%d}_WEW_top_down_sample_{i}"
    write_artifacts(run_name, forecast, model, pj, backtest_specs)

In [ ]:
pd.set_option("display.max_columns", 130)
train_data[0].head()

## Each customer seperately

In [ ]:
from tqdm.notebook import tqdm
import openstef.metrics.metrics as metrics

In [ ]:
# Run backtest
# Calculate results immediately.
combined_forecast = pd.DataFrame(columns=['forecast', 'realised'])
res_metrics=pd.DataFrame(columns=['rmae_lowest', 'rmae', 'rmae_highest', 'rmse', 'mae'])
for customer in tqdm(carm_measurements.columns):
    if customer == 'Total':
        continue
    input_data = pd.DataFrame(dict(load=carm_measurements[customer])).merge(predictors, left_index=True, right_index=True, how='inner')
    # Perform the backtest
    forecast, models, train_data, validation_data, test_data = train_model_and_forecast_back_test(
        pj,
        modelspecs = modelspecs,
        input_data = input_data,
        **backtest_specs,
    )
    
    # If n_folds > 1, models is a list of models. In that case, only use the first model.
    if backtest_specs['n_folds'] > 1:
        model=models[0]
    else:
        model=models
        
    # Check if actually a forecast was made
    if forecast["forecast"].isna().sum()<10:
        print('No forecast was made. Perhaps the load was constant (0) or missing?')
        continue
        
    # Calculate KPI
    res_metrics.loc[customer, :] = [
        metrics.r_mae_lowest(forecast["realised"], forecast["forecast"]),
        metrics.r_mae(forecast["realised"], forecast["forecast"]),
        metrics.r_mae_highest(forecast["realised"], forecast["forecast"]),
        metrics.rmse(forecast["realised"], forecast["forecast"]),
        metrics.mae(forecast["realised"], forecast["forecast"]),
                            ]
    
    if len(combined_forecast) == 0:
        combined_forecast = forecast[['forecast','realised']]
    else:
        combined_forecast['forecast'] += forecast['forecast']
        combined_forecast['realised'] += forecast['realised']

    run_name = f"{datetime.utcnow():%Y%m%d}_WEW_customer_{customer}"
    write_artifacts(run_name, forecast, model, pj, backtest_specs)

    
combined_forecast.to_csv(f"output/{datetime.utcnow():%Y%m%d}_WEW_total_customers.csv")
res_metrics.to_csv(f"output/{datetime.utcnow():%Y%m%d}_WEW_total_customers_metrics.csv")


In [ ]:
# load from history - useful for when exporting the notebook went wrong and you don't want to redo the entire backtests
combined_forecast = pd.read_csv('output/20230731_WEW_total_customers.csv', index_col=0, parse_dates=True)
res_metrics = pd.read_csv('output/20230731_WEW_total_customers_metrics.csv', index_col=0, parse_dates=True)

## Evaluate results

In [ ]:
# Retrieve the forecasts from the stored files
import glob
relevant_files = glob.glob('output/20230731_WEW_customer_*/forecast.csv')
sum_forecast = pd.DataFrame()
sum_realised = pd.DataFrame()
for file in tqdm(relevant_files):
    fc = pd.read_csv(file, index_col=0, parse_dates=True, compression='gzip')

    if len(sum_forecast)==0:
        sum_forecast=fc[['forecast']]
        sum_realised=fc[['realised']]
        continue

    sum_forecast = pd.DataFrame(sum_forecast).merge(fc.forecast, left_index=True, right_index=True, how='outer').sum(axis=1)
    sum_realised = pd.DataFrame(sum_realised).merge(fc.realised, left_index=True, right_index=True, how='outer').sum(axis=1)
                              
    if sum_forecast.isna().sum()>1000:
        print('NAN!!')
        break

In [ ]:
aggregate = pd.DataFrame(data=dict(forecast=sum_forecast, realised=sum_realised))
res_metrics.loc['Aggregate',:] = [
        metrics.r_mae_lowest(aggregate["realised"], aggregate["forecast"]),
        metrics.r_mae(aggregate["realised"], aggregate["forecast"]),
        metrics.r_mae_highest(aggregate["realised"], aggregate["forecast"]),
        metrics.rmse(aggregate["realised"], aggregate["forecast"]),
        metrics.mae(aggregate["realised"], aggregate["forecast"]),
                            ]


In [ ]:
direct_forecast = pd.read_csv('output/20230731_WEW_top_down_sample_0/forecast.csv', compression='gzip', index_col=0, parse_dates=True)

res_metrics.loc['Direct', :] = [
        metrics.r_mae_lowest(direct_forecast["realised"], direct_forecast["forecast"]),
        metrics.r_mae(direct_forecast["realised"], direct_forecast["forecast"]),
        metrics.r_mae_highest(direct_forecast["realised"], direct_forecast["forecast"]),
        metrics.rmse(direct_forecast["realised"], direct_forecast["forecast"]),
        metrics.mae(direct_forecast["realised"], direct_forecast["forecast"]),
                            ]


In [ ]:
fig = res_metrics.loc[['Direct','Aggregate'],'mae'].plot(kind='bar', width=600)
fig.update_layout(dict(yaxis=dict(title='MAE [MW]'), xaxis=dict(title=''), showlegend=False), title='Forecasting the total customer load directly <br>versus <br>aggregating the forecasts for each individual customer',
                  margin=dict(t=100, b=0, l=0, r=0))

In [ ]:
# Evaluate MAE of largest customers
res_metrics['shortname'] = [x if len(x)<10 else f'{x[:8]}..' for x in res_metrics.index ]
fig = res_metrics.sort_values(by='mae', ascending=False)[:10].plot(y='mae', x='shortname', width=600, kind='bar')
fig.update_layout(dict(yaxis=dict(title='MAE [MW]'), xaxis=dict(title='')))

The MAE of forecasting the direct load is larger than the MAE of the aggregate of the forecasts of the individual customers. 

This analysis should be repeated to claim statistical significance. However, it can be seen that forecasting the individual customers seperately at least does not increase the forecasting error. 

Moreover, the two customers with the largest MAE are the large wind and solar park, as depicted in a previous figure. 

From a machine learning point of view, it can be understood that forecasting the loads of a solar and windpark seperately, allows machine learning models to learn a more straightforward relation compared to forecasting the aggregate of that. 

In [ ]:
# Let's compare the direct and aggregated forecasts and realised
combined = direct_forecast[['forecast','realised']].rename(columns=dict(forecast='forecast_direct', realised='realised_direct'))
combined = combined.merge(aggregate.rename(columns=dict(forecast='forecast_aggregate', realised='realised_aggregate')), left_index=True, right_index=True, how='outer')
fig = combined.plot()
fig.update_layout(yaxis=dict(title='Power [MW]'), xaxis=dict(title=''))

Figure shows the timeseries of the direct and aggregated forecast, as well as the realization. As a sanity check, the individual realizations of all customers from the backtest dataframes are aggregated to show it matches the original `total_customers`.

# Export notebook as html
Write this notebook to html.

In [ ]:

nb_fname = '43.Compare_SumForecastsCustomers_vs_ForecastSumCustomers'
command=f"jupyter nbconvert {nb_fname}.ipynb --to html --no-input --output results/{nb_fname}.html"
print(f"Command to be executed: {command}.")
os.system(command)